In [1]:
#filename = "ml.py"
#exec(compile(open(filename, "rb").read(), filename, 'exec'))

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import dateutil.parser as parser
from sklearn.linear_model import LogisticRegression

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from sklearn.cluster import KMeans


nltk.download('vader_lexicon')

# read the dataset into pandas dataframe
# df = pd.read_csv('./../downsampled_data', delim_whitespace=False).dropna()
df = pd.read_csv('./../downsampled_data.csv').dropna()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/trevorcarpenter/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
df.columns

Index(['Unnamed: 0', 'video_id', 'title', 'publishedAt', 'channelId',
       'channelTitle', 'categoryId', 'duration', 'definition', 'trended_later',
       'time_retrieved', 'tags', 'thumbnail_link', 'ratings_disabled',
       'description', 'Channel_viewCount', 'Channel_subscriberCount',
       'Channel_hiddenSubscriberCount', 'Channel_videoCount', 'Channel_title',
       'Channel_description', 'Channel_publishedAt', 'Channel_country',
       'view_count_update_11_19_14', 'likes_update_11_19_14',
       'dislikes_update_11_19_14', 'comment_count_update_11_19_14',
       '11_19_14_update_timestamp', 'trending?', 'engagement_rate', 'INTL',
       'UNK', 'USA'],
      dtype='object')

In [5]:
df['categoryId'] = df['categoryId'].astype(np.float)

categories = pd.DataFrame(np.zeros((len(df), len(df['categoryId'].unique()))), columns = df['categoryId'].unique())
index = 0
for id in df['categoryId']:
    categories[id][index] = 1
    index += 1

In [6]:
# display the first five rows
retrieval_time = df['time_retrieved']
publish_time = df['publishedAt']
channel_publish_time = df['Channel_publishedAt']
retrieval_time_11_19_14 = df['11_19_14_update_timestamp']
columns_to_drop = ['Unnamed: 0', 'definition', 'categoryId', 'publishedAt', 'time_retrieved', 'Channel_title', '11_19_14_update_timestamp', 'Channel_publishedAt', 'video_id', 'channelId', 'thumbnail_link', 'Channel_country']
df = df.drop(columns_to_drop, axis = 1)

In [7]:
categories.columns

Float64Index([24.0, 25.0, 26.0, 17.0, 22.0, 20.0,  1.0, 10.0, 23.0, 28.0,  2.0,
              27.0, 19.0, 15.0, 29.0, 30.0],
             dtype='float64')

In [8]:
index = 0
for column in categories.columns:
    df.insert(df.shape[1], "Category_"+str(column), categories[column])
    index += 1

In [9]:
## time updates
import dateutil.parser as parser
age = []
age_update = []
channel_age = []
for i in df.index:
    channel_publish_time[i] = channel_publish_time[i].replace("\"", "")
    age.append(parser.isoparse(retrieval_time[i]) - parser.isoparse(publish_time[i]))
    age_update.append(parser.isoparse(retrieval_time_11_19_14[i]) - parser.isoparse(publish_time[i]))
    channel_age.append(parser.isoparse(channel_publish_time[i]) - parser.isoparse(publish_time[i]))
    if df['ratings_disabled'][i] == 'True':
        df['ratings_disabled'][i] = True
    elif df['ratings_disabled'][i] == 'False':
        df['ratings_disabled'][i] = False
        
    if df['Channel_hiddenSubscriberCount'][i] == 'True':
        df['Channel_hiddenSubscriberCount'][i] = True
    elif df['Channel_hiddenSubscriberCount'][i] == 'False':
        df['Channel_hiddenSubscriberCount'][i] = False
        
        
    if df['trended_later'][i] == 'True':
        df['trended_later'][i] = True
    elif df['trended_later'][i] == 'False':
        df['trended_later'][i] = False

/Applications/anaconda2/envs/newEnv/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Applications/anaconda2/envs/newEnv/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda2/envs/newEnv/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda2/envs/newEn

In [10]:
## sentiment values

titles = df['title']
channel_title = df['channelTitle']
description = df['description']
channel_description = df['Channel_description']

title_sentiment_vals = []
channel_title_sentiment_vals = []
description_sentiment_vals = []
channel_description_sentiment_vals = []

sid = SentimentIntensityAnalyzer()
for sentence in titles:
    ss = sid.polarity_scores(str(sentence))
    title_sentiment_vals.append(ss['pos']-ss['neg'])
    
for sentence in channel_title:
    ss = sid.polarity_scores(str(sentence))
    channel_title_sentiment_vals.append(ss['pos']-ss['neg'])
    
for sentence in description:
    ss = sid.polarity_scores(str(sentence))
    description_sentiment_vals.append(ss['pos']-ss['neg'])
    
for sentence in channel_description:
    ss = sid.polarity_scores(str(sentence))
    channel_description_sentiment_vals.append(ss['pos']-ss['neg'])

In [11]:
df

,title,channelTitle,duration,trended_later,tags,ratings_disabled,description,Channel_viewCount,Channel_subscriberCount,Channel_hiddenSubscriberCount,...,Category_1.0,Category_10.0,Category_23.0,Category_28.0,Category_2.0,Category_27.0,Category_19.0,Category_15.0,Category_29.0,Category_30.0
0,Biden Victory Cold Open - SNL,Saturday Night Live,-0.0707230866960673,False,36.0,False,Joe Biden (Jim Carrey) and Donald Trump (Alec ...,0.819940,0.504608,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,How President Trump and the White House reacte...,CNBC Television,-0.0856798827419832,False,28.0,False,CNBC's Kayla Tausche joins The News with Shepa...,-0.274970,-0.350255,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Watch Live: President-Elect Joe Biden Addresse...,ABC News,0.09488158288091483,False,13.0,False,#Election2020 #Election #JoeBiden #PresidentEl...,0.650191,0.461214,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,I GOT PLASTIC SURGERY TO SEE HOW MY FRIENDS RE...,Alexa Rivera,-0.07104706061763949,False,0.0,False,"This was definitely such a fun prank, their re...",-0.258217,0.002973,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,FC Barcelona 5 - 2 Real Betis - HIGHLIGHTS & G...,beIN SPORTS USA,-0.07288291283988188,False,12.0,False,Lionel Messi entró al segundo tiempo y resucit...,-0.330711,-0.407708,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475,VIRTUAL DERVISH EXPERIENCE FRAGMAN,hodjapasha,-0.09566907865712558,False,0.0,False,THE SEMA CEREMONY IS A SPRITUAL JOURNEY OF DER...,-0.341435,-0.441365,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1476,Joe Biden beats Donald Trump to win US preside...,DW News,0.011674280690458535,False,7.0,False,Democratic candidate Joe Biden has won the US ...,-0.281025,-0.281692,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1477,LEE SUHYUN - &#39;ALIEN&#39; PERFORMANCE VIDEO...,AKMU,-0.08692178277467658,False,21.0,False,이수현 #LEESUHYUN #1stSingle #ALIEN #Dance_Perfor...,-0.295373,-0.265202,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1478,Iyanla: People Have Failed You | Iyanla: Fix M...,OWN,-0.09053949156556598,False,24.0,False,Christopher and Amber tell Iyanla the painful ...,-0.080214,-0.051704,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
from sklearn import preprocessing

df['title'] = title_sentiment_vals
df['channelTitle'] = channel_title_sentiment_vals
df['description'] = description_sentiment_vals
df['Channel_description'] = channel_description_sentiment_vals

le = preprocessing.LabelEncoder()
df['trending?'] = le.fit_transform(df['trending?'])
df['ratings_disabled'] = le.fit_transform(df['ratings_disabled'])
df['Channel_hiddenSubscriberCount'] = le.fit_transform(df['Channel_hiddenSubscriberCount'])
pd.set_option('display.max_columns', None)
df = df.dropna()

In [84]:
# dummy values for now - get these interactively later...
import tensorflow as tf

best_model = False
if best_model:
    num_hidden_layers = 3
    num_hidden_layer_nodes = [20, 30, 20]
    train_ratio = .7
    hidden_layer_activations = ['sigmoid', 'sigmoid', 'sigmoid', 'sigmoid']
    optimizer = 'adam'
    learning_rate = .005
    loss = 'mean_squared_error'
    metrics = [tf.keras.metrics.Accuracy(),tf.keras.metrics.Recall(),tf.keras.metrics.Precision()]
    metrics_names = ["accuracy","recall","precision"]
    epochs = 300
    batch_size = 200
else:
    # build a custom model
    num_hidden_layers = 3
    num_hidden_layer_nodes = [20, 30, 20]
    train_ratio = .7
    hidden_layer_activations = ['sigmoid', 'sigmoid', 'sigmoid', 'sigmoid']
    optimizer = 'adam'
    learning_rate = .005
    loss = 'mean_squared_error'
    metrics = [tf.keras.metrics.Accuracy(),tf.keras.metrics.Recall(),tf.keras.metrics.Precision()]
    metrics_names = ["accuracy","recall","precision"]
    epochs = 300
    batch_size = 200

In [85]:
# Split the data into training and testing set by 70:30
ratio = 0.7
train, test = train_test_split(df, train_size=ratio, random_state=42)
train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

In [86]:
# separate data into x and y - just random y for now..
train_X = train.loc[:,train.columns != 'trending?']
train_Y = train['trending?']
test_X = test.loc[:,test.columns != 'trending?']
test_Y = test['trending?']

train_Y = KMeans(n_clusters=3).fit(train_X)
df['trending?'] = labels

In [87]:
# build the logistic regression model - need clean data...
from sklearn.linear_model import LogisticRegression
LR_model = LogisticRegression(multi_class='ovr')
LR_model.fit(train_X, train_Y)

/Applications/anaconda2/envs/newEnv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Applications/anaconda2/envs/newEnv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/sta

LogisticRegression(multi_class='ovr')

In [88]:
# confusion matrix
from sklearn.metrics import confusion_matrix

pred_Y = LR_model.predict(test_X);
confusion_matrix(test_Y, pred_Y)

array([[130,   1,   0],
       [  2, 167,   1],
       [  0,   0,  89]])

In [89]:
# evaluate the accuracy of the LR model
accuracy = LR_model.score(test_X, test_Y)
print(accuracy)

0.9897435897435898


In [99]:
# build the ANN model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input

ANN_model = keras.Sequential()

# add hidden layers
for i in range(num_hidden_layers):
    if i == 0:
        ANN_model.add(Dense(num_hidden_layer_nodes[i], input_dim = train_X.shape[1], activation=hidden_layer_activations[i]))
    ANN_model.add(Dense(num_hidden_layer_nodes[i], activation=hidden_layer_activations[i]))

# add output layers
ANN_model.add(Dense(4, activation=hidden_layer_activations[num_hidden_layers]))

ANN_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
ANN_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 20)                740       
_________________________________________________________________
dense_21 (Dense)             (None, 20)                420       
_________________________________________________________________
dense_22 (Dense)             (None, 30)                630       
_________________________________________________________________
dense_23 (Dense)             (None, 20)                620       
_________________________________________________________________
dense_24 (Dense)             (None, 4)                 84        
Total params: 2,494
Trainable params: 2,494
Non-trainable params: 0
_________________________________________________________________


In [100]:
ty = np.zeros((len(train_Y), 4))
for index in range(len(train_Y)):
    if train_Y[index] == 0:
        ty[index] = [1,0,0,0]
    elif train_Y[index] == 1:
        ty[index] = [0,1,0,0]
    elif train_Y[index] == 2:
        ty[index] = [0,0,1,0]
    elif train_Y[index] == 3:
        ty[index] = [0,0,0,1]
        
        
testy = np.zeros((len(test_Y), 4))
for index in range(len(test_Y)):
    if test_Y[index] == 0:
        testy[index] = [1,0,0,0]
    elif test_Y[index] == 1:
        testy[index] = [0,1,0,0]
    elif test_Y[index] == 2:
        testy[index] = [0,0,1,0]
    elif test_Y[index] == 3:
        testy[index] = [0,0,0,1]

In [101]:
# train the model
ANN_model.fit(train_X, ty, validation_data=(test_X, testy), epochs=epochs, batch_size=batch_size)

Train on 908 samples, validate on 390 samples
Epoch 1/300
908/908 [==============================] - 1s 1ms/sample - loss: 0.3010 - accuracy: 0.0000e+00 - recall_1: 0.9989 - precision_1: 0.3330 - val_loss: 0.2925 - val_accuracy: 0.0000e+00 - val_recall_1: 1.0000 - val_precision_1: 0.3333
Epoch 2/300
908/908 [==============================] - 0s 79us/sample - loss: 0.2866 - accuracy: 0.0000e+00 - recall_1: 0.9989 - precision_1: 0.3330 - val_loss: 0.2783 - val_accuracy: 0.0000e+00 - val_recall_1: 1.0000 - val_precision_1: 0.3333
Epoch 3/300
908/908 [==============================] - 0s 84us/sample - loss: 0.2727 - accuracy: 0.0000e+00 - recall_1: 0.9989 - precision_1: 0.3330 - val_loss: 0.2647 - val_accuracy: 0.0000e+00 - val_recall_1: 1.0000 - val_precision_1: 0.3333
Epoch 4/300
908/908 [==============================] - 0s 82us/sample - loss: 0.2595 - accuracy: 0.0000e+00 - recall_1: 0.8645 - precision_1: 0.3232 - val_loss: 0.2520 - val_accuracy: 0.0000e+00 - val_recall_1: 0.5641 - val

908/908 [==============================] - 0s 72us/sample - loss: 0.1644 - accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00 - val_loss: 0.1647 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 33/300
908/908 [==============================] - 0s 66us/sample - loss: 0.1641 - accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00 - val_loss: 0.1645 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 34/300
908/908 [==============================] - 0s 85us/sample - loss: 0.1639 - accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00 - val_loss: 0.1642 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 35/300
908/908 [==============================] - 0s 87us/sample - loss: 0.1636 - accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00 - val_loss: 0.1640 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00

908/908 [==============================] - 0s 85us/sample - loss: 0.1541 - accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00 - val_loss: 0.1547 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 64/300
908/908 [==============================] - 0s 76us/sample - loss: 0.1532 - accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00 - val_loss: 0.1537 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 65/300
908/908 [==============================] - 0s 107us/sample - loss: 0.1521 - accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00 - val_loss: 0.1526 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00
Epoch 66/300
908/908 [==============================] - 0s 110us/sample - loss: 0.1509 - accuracy: 0.0000e+00 - recall_1: 0.0000e+00 - precision_1: 0.0000e+00 - val_loss: 0.1515 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0000e+

908/908 [==============================] - 0s 116us/sample - loss: 0.0948 - accuracy: 0.0000e+00 - recall_1: 0.7500 - precision_1: 0.7747 - val_loss: 0.0994 - val_accuracy: 0.0000e+00 - val_recall_1: 0.7026 - val_precision_1: 0.7446
Epoch 97/300
908/908 [==============================] - 0s 96us/sample - loss: 0.0936 - accuracy: 0.0000e+00 - recall_1: 0.7522 - precision_1: 0.7753 - val_loss: 0.0983 - val_accuracy: 0.0000e+00 - val_recall_1: 0.7051 - val_precision_1: 0.7453
Epoch 98/300
908/908 [==============================] - 0s 94us/sample - loss: 0.0925 - accuracy: 0.0000e+00 - recall_1: 0.7555 - precision_1: 0.7769 - val_loss: 0.0972 - val_accuracy: 0.0000e+00 - val_recall_1: 0.7128 - val_precision_1: 0.7514
Epoch 99/300
908/908 [==============================] - 0s 84us/sample - loss: 0.0915 - accuracy: 0.0000e+00 - recall_1: 0.7533 - precision_1: 0.7773 - val_loss: 0.0962 - val_accuracy: 0.0000e+00 - val_recall_1: 0.7179 - val_precision_1: 0.7527
Epoch 100/300
908/908 [=========

908/908 [==============================] - 0s 83us/sample - loss: 0.0773 - accuracy: 0.0000e+00 - recall_1: 0.7720 - precision_1: 0.7850 - val_loss: 0.0835 - val_accuracy: 0.0000e+00 - val_recall_1: 0.7436 - val_precision_1: 0.7612
Epoch 130/300
908/908 [==============================] - 0s 69us/sample - loss: 0.0771 - accuracy: 0.0000e+00 - recall_1: 0.7720 - precision_1: 0.7850 - val_loss: 0.0834 - val_accuracy: 0.0000e+00 - val_recall_1: 0.7436 - val_precision_1: 0.7572
Epoch 131/300
908/908 [==============================] - 0s 69us/sample - loss: 0.0769 - accuracy: 0.0000e+00 - recall_1: 0.7731 - precision_1: 0.7852 - val_loss: 0.0831 - val_accuracy: 0.0000e+00 - val_recall_1: 0.7436 - val_precision_1: 0.7612
Epoch 132/300
908/908 [==============================] - 0s 70us/sample - loss: 0.0767 - accuracy: 0.0000e+00 - recall_1: 0.7731 - precision_1: 0.7852 - val_loss: 0.0830 - val_accuracy: 0.0000e+00 - val_recall_1: 0.7436 - val_precision_1: 0.7572
Epoch 133/300
908/908 [=======

908/908 [==============================] - 0s 98us/sample - loss: 0.0709 - accuracy: 0.0000e+00 - recall_1: 0.7819 - precision_1: 0.7863 - val_loss: 0.0774 - val_accuracy: 0.0000e+00 - val_recall_1: 0.7513 - val_precision_1: 0.7571
Epoch 163/300
908/908 [==============================] - 0s 92us/sample - loss: 0.0707 - accuracy: 0.0000e+00 - recall_1: 0.7819 - precision_1: 0.7863 - val_loss: 0.0772 - val_accuracy: 0.0000e+00 - val_recall_1: 0.7462 - val_precision_1: 0.7558
Epoch 164/300
908/908 [==============================] - 0s 104us/sample - loss: 0.0704 - accuracy: 0.0000e+00 - recall_1: 0.7808 - precision_1: 0.7852 - val_loss: 0.0770 - val_accuracy: 0.0000e+00 - val_recall_1: 0.7462 - val_precision_1: 0.7558
Epoch 165/300
908/908 [==============================] - 0s 50us/sample - loss: 0.0701 - accuracy: 0.0000e+00 - recall_1: 0.7808 - precision_1: 0.7852 - val_loss: 0.0767 - val_accuracy: 0.0000e+00 - val_recall_1: 0.7462 - val_precision_1: 0.7558
Epoch 166/300
908/908 [======

908/908 [==============================] - 0s 56us/sample - loss: 0.0469 - accuracy: 0.0000e+00 - recall_1: 0.9802 - precision_1: 0.8927 - val_loss: 0.0506 - val_accuracy: 0.0000e+00 - val_recall_1: 0.9692 - val_precision_1: 0.8936
Epoch 196/300
908/908 [==============================] - 0s 73us/sample - loss: 0.0457 - accuracy: 0.0000e+00 - recall_1: 0.9791 - precision_1: 0.9099 - val_loss: 0.0497 - val_accuracy: 0.0000e+00 - val_recall_1: 0.9667 - val_precision_1: 0.8976
Epoch 197/300
908/908 [==============================] - 0s 90us/sample - loss: 0.0447 - accuracy: 0.0000e+00 - recall_1: 0.9692 - precision_1: 0.9110 - val_loss: 0.0484 - val_accuracy: 0.0000e+00 - val_recall_1: 0.9667 - val_precision_1: 0.9173
Epoch 198/300
908/908 [==============================] - 0s 89us/sample - loss: 0.0437 - accuracy: 0.0000e+00 - recall_1: 0.9835 - precision_1: 0.9420 - val_loss: 0.0470 - val_accuracy: 0.0000e+00 - val_recall_1: 0.9744 - val_precision_1: 0.9383
Epoch 199/300
908/908 [=======

908/908 [==============================] - 0s 77us/sample - loss: 0.0199 - accuracy: 0.0000e+00 - recall_1: 0.9945 - precision_1: 0.9945 - val_loss: 0.0236 - val_accuracy: 0.0000e+00 - val_recall_1: 0.9769 - val_precision_1: 0.9744
Epoch 229/300
908/908 [==============================] - 0s 78us/sample - loss: 0.0194 - accuracy: 0.0000e+00 - recall_1: 0.9956 - precision_1: 0.9945 - val_loss: 0.0229 - val_accuracy: 0.0000e+00 - val_recall_1: 0.9872 - val_precision_1: 0.9796
Epoch 230/300
908/908 [==============================] - 0s 66us/sample - loss: 0.0189 - accuracy: 0.0000e+00 - recall_1: 0.9967 - precision_1: 0.9956 - val_loss: 0.0226 - val_accuracy: 0.0000e+00 - val_recall_1: 0.9769 - val_precision_1: 0.9820
Epoch 231/300
908/908 [==============================] - 0s 68us/sample - loss: 0.0184 - accuracy: 0.0000e+00 - recall_1: 0.9967 - precision_1: 0.9956 - val_loss: 0.0221 - val_accuracy: 0.0000e+00 - val_recall_1: 0.9846 - val_precision_1: 0.9771
Epoch 232/300
908/908 [=======

908/908 [==============================] - 0s 96us/sample - loss: 0.0105 - accuracy: 0.0000e+00 - recall_1: 0.9978 - precision_1: 0.9978 - val_loss: 0.0151 - val_accuracy: 0.0000e+00 - val_recall_1: 0.9744 - val_precision_1: 0.9744
Epoch 262/300
908/908 [==============================] - 0s 67us/sample - loss: 0.0105 - accuracy: 0.0000e+00 - recall_1: 0.9956 - precision_1: 0.9945 - val_loss: 0.0147 - val_accuracy: 0.0000e+00 - val_recall_1: 0.9795 - val_precision_1: 0.9845
Epoch 263/300
908/908 [==============================] - 0s 70us/sample - loss: 0.0101 - accuracy: 0.0000e+00 - recall_1: 0.9978 - precision_1: 0.9989 - val_loss: 0.0149 - val_accuracy: 0.0000e+00 - val_recall_1: 0.9769 - val_precision_1: 0.9744
Epoch 264/300
908/908 [==============================] - 0s 67us/sample - loss: 0.0100 - accuracy: 0.0000e+00 - recall_1: 0.9978 - precision_1: 0.9978 - val_loss: 0.0145 - val_accuracy: 0.0000e+00 - val_recall_1: 0.9821 - val_precision_1: 0.9795
Epoch 265/300
908/908 [=======

908/908 [==============================] - 0s 78us/sample - loss: 0.0066 - accuracy: 0.0000e+00 - recall_1: 0.9978 - precision_1: 0.9989 - val_loss: 0.0120 - val_accuracy: 0.0000e+00 - val_recall_1: 0.9744 - val_precision_1: 0.9769
Epoch 295/300
908/908 [==============================] - 0s 77us/sample - loss: 0.0067 - accuracy: 0.0000e+00 - recall_1: 0.9989 - precision_1: 0.9989 - val_loss: 0.0120 - val_accuracy: 0.0000e+00 - val_recall_1: 0.9744 - val_precision_1: 0.9744
Epoch 296/300
908/908 [==============================] - 0s 87us/sample - loss: 0.0065 - accuracy: 0.0000e+00 - recall_1: 0.9978 - precision_1: 0.9967 - val_loss: 0.0122 - val_accuracy: 0.0000e+00 - val_recall_1: 0.9718 - val_precision_1: 0.9718
Epoch 297/300
908/908 [==============================] - 0s 90us/sample - loss: 0.0065 - accuracy: 0.0000e+00 - recall_1: 0.9989 - precision_1: 0.9989 - val_loss: 0.0119 - val_accuracy: 0.0000e+00 - val_recall_1: 0.9744 - val_precision_1: 0.9769
Epoch 298/300
908/908 [=======

In [102]:
# confusion matrix
pred_Y = ANN_model.predict_classes(test_X);
confusion_matrix(test_Y, pred_Y)

array([[130,   1,   0],
       [  2, 164,   4],
       [  0,   2,  87]])

In [76]:
# report evaluation metrics 
evaluated_metrics = ANN_model.evaluate(test_X, test_Y)
for i in range(len(metrics)):
    print(metrics_names[i] + ": %.2f" % evaluated_metrics[i])

390/390 [==============================] - 0s 249us/sample - loss: 0.2383 - accuracy: 0.0000e+00 - recall: 0.9923 - precision: 0.9961
accuracy: 0.24
recall: 0.00
precision: 0.99


In [23]:
len(df)

1298

In [31]:
tr = df['trending?']
df_adjust = df.drop(['trending?', 'trended_later'], axis = 1)

In [32]:
df_adjust

,title,channelTitle,duration,tags,ratings_disabled,description,Channel_viewCount,Channel_subscriberCount,Channel_hiddenSubscriberCount,Channel_videoCount,Channel_description,view_count_update_11_19_14,likes_update_11_19_14,dislikes_update_11_19_14,comment_count_update_11_19_14,engagement_rate,INTL,UNK,USA,Category_24.0,Category_25.0,Category_26.0,Category_17.0,Category_22.0,Category_20.0,Category_1.0,Category_10.0,Category_23.0,Category_28.0,Category_2.0,Category_27.0,Category_19.0,Category_15.0,Category_29.0,Category_30.0
0,0.000,0.0,-0.0707230866960673,36.0,0,0.034,0.819940,0.504608,0,-0.190170,0.122,6.152175,3.391273,10.316363,2.855296,0.028492,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000,0.0,-0.0856798827419832,28.0,0,0.048,-0.274970,-0.350255,0,1.155772,0.000,3.001911,0.578677,4.999903,3.142250,0.090361,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000,0.0,0.09488158288091483,13.0,0,0.062,0.650191,0.461214,0,1.266836,0.107,1.368339,0.537524,2.314506,1.066382,0.006721,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.274,0.0,-0.07104706061763949,0.0,0,0.276,-0.258217,0.002973,0,-0.408313,0.000,1.882086,2.655548,2.774959,0.721674,0.045112,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000,0.0,-0.07288291283988188,12.0,0,0.000,-0.330711,-0.407708,0,-0.204876,0.188,0.280922,-0.071268,0.049159,0.022729,0.029918,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384,0.000,0.0,-0.04518314254546002,20.0,0,0.000,-0.264347,-0.056911,0,-0.387676,0.000,0.212355,0.013470,0.244207,0.048079,0.004168,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1385,0.447,0.0,0.005464780526991646,26.0,0,0.383,-0.317810,-0.380631,0,-0.384501,0.201,-0.211730,-0.188659,-0.189775,-0.123655,0.001349,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1386,0.000,0.0,0.13068070121464131,37.0,0,0.161,-0.338899,-0.416908,0,-0.404090,0.129,-0.199097,-0.142528,-0.196044,-0.075337,0.017865,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1387,0.000,0.0,-0.08233215221907061,16.0,0,0.120,14.752324,13.444714,0,0.030849,0.067,2.555740,2.090946,1.436440,0.510866,0.001155,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


array([2, 1, 0, ..., 2, 1, 1], dtype=int32)

/Applications/anaconda2/envs/newEnv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
